In [21]:
import os
import re
import time
import folderstats
import chardet
#import docx
import textract
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
data_folder = os.listdir('../data/t5')

In [13]:
files = {}

for ext in {re.search('(\.\w+)', file).group(0) for file in data_folder if (file != '.DS_Store')}:
    filenames = [f for f in data_folder if f.endswith(ext)]
    files[ext] = {
        'count': len(filenames),
        'filenames': filenames
    }

In [14]:
[(key, files[key]['count']) for key in files.keys()]

[('.pdf', 1073),
 ('.xls', 250),
 ('.text', 711),
 ('.doc', 533),
 ('.html', 1093),
 ('.jpg', 362),
 ('.gif', 67),
 ('.ppt', 368)]

In [15]:
files.keys()

dict_keys(['.pdf', '.xls', '.text', '.doc', '.html', '.jpg', '.gif', '.ppt'])

In [ ]:
files['.pdf', '.xls', ]

In [23]:
with open('../data/t5/000077.html', 'r') as file:
    test_html = file.readlines()

'<html>\n <head>\n <title>1997 Economic Census:  ZIP Code Statistics\n  - 98589\n </title>\n </head>\n \n <body bgcolor=#ffffff vlink=#66330>\n <a href="http://www.census.gov">\n <img src="http://www.census.gov/main/www/img/cb_head.gif" border="0" alt="U.S. Census Bureau"></a>\n <table width="100%"><tr><td rowspan="3" align="left">\n <a href="/epcd/www/econ97.html"><img src="../../www/img/ec97_76.gif" border="0" alt="1997 Economic Census main page"></a></td>\n <td rowspan="3"><b><font size="+2">1997 Economic Census: \n <br>  ZIP Code Statistics\n <br>98589\n &nbsp; </font> &nbsp;   (Tenino, WA)\n </b></td>\n \n <td align="right" valign="bottom">  <FONT FACE="Arial, Helvetica"><font size=2>\n <FORM ACTION="http://www.census.gov/cgi-bin/geo/location" METHOD="POST">\n <select name="Location">\n  <option value="/epcd/ec97zip/us/US000.HTM">Select a state\n </option> \n <option value="/epcd/ec97zip/us/US00000.HTM">\n United States</option> <option value="/epcd/ec97zip/al/AL00000.HTM">\n Alab

In [28]:
soup = BeautifulSoup(' '.join(test_html), 'html.parser')

In [34]:
soup.find('body').text

'\n\n\n\n\n1997 Economic Census: \n   ZIP Code Statistics\n 98589\n \xa0  \xa0   (Tenino, WA)\n \n \n\n\nSelect a state\n \n\n United States \n Alabama \n Alaska \n Arizona \n Arkansas \n California \n Colorado \n Connecticut \n Delaware \n District of Columbia \n Florida \n Georgia \n Hawaii \n Idaho \n Illinois \n Indiana \n Iowa \n Kansas \n Kentucky \n Louisiana \n Maine \n Maryland \n Massachusetts \n Michigan \n Minnesota \n Mississippi \n Missouri \n Montana \n Nebraska \n Nevada \n New Hampshire \n New Jersey \n New Mexico \n New York \n North Carolina \n North Dakota \n Ohio \n Oklahoma \n Oregon \n Pennsylvania \n Rhode Island \n South Carolina \n South Dakota \n Tennessee \n Texas \n United States \n Utah \n Vermont \n Virginia \n Washington \n West Virginia \n Wisconsin \n Wyoming\n\n\n\n\n \n\n\nSelect 1st 3 digits of ZIP in WA\n \n980\n981\n982\n983\n984\n985\n986\n988\n989\n990\n991\n992\n993\n994\n999\n\n\n\n\n'

In [6]:
df = folderstats.folderstats('t5/', ignore_hidden=True)
df.head()

,id,path,name,extension,size,atime,mtime,ctime,folder,num_files,depth,parent,uid
0,2,t5/000001.doc,000001,doc,0,2020-09-02 12:08:42,2020-09-02 12:08:29,2020-09-02 12:08:29,False,NaN,0,1,501
1,3,t5/000002.doc,000002,doc,57856,2020-08-27 22:13:50,2011-02-08 15:37:40,2020-08-27 22:13:40,False,NaN,0,1,501
2,4,t5/000003.doc,000003,doc,55808,2020-09-02 10:27:33,2011-02-08 15:37:40,2020-08-27 22:13:40,False,NaN,0,1,501
3,5,t5/000004.doc,000004,doc,175616,2020-08-27 22:13:50,2011-02-08 15:37:40,2020-08-27 22:13:40,False,NaN,0,1,501
4,6,t5/000005.doc,000005,doc,180736,2020-08-27 22:13:50,2011-02-08 15:37:40,2020-08-27 22:13:40,False,NaN,0,1,501


In [7]:
df.shape

(4458, 13)

In [13]:
for _ in os.listdir('t5/')[1:10]:
    print(_)
    try:
        print(textract.process('t5/' + _)[:200])
    except:
        print('')

000001.doc

000002.doc
b'\n[pic]\n\nRelease No. 0043.04\nOffice of Communications (202) 720-4623\n\n                                 BSE Update \xe2\x80\x94 January 27, 2004\n\n\n\n\nDepopulation Activities\n\n     On Monday, Jan. 26, 2004, 18 ani'
000003.doc
b'\n                                    [pic]\n\nRelease No. 0048.04\nOffice of Communications (202) 720-4623\n\n                                 BSE Update \xe2\x80\x93 Wednesday, January 28, 2004\n\nDepopulation Activ'
000004.doc
b'\n\n|Cascade bilberry                 |\n|Vaccinium deliciosum Piper       |\n|Plant Symbol = VADE              |\n\nContributed by: USDA NRCS National Plant Data Center & Oregon Plant\nMaterials Center\n\n\n\n\n'
000005.doc
b'\n\n|black huckleberry                |\n|Vaccinium membranaceum Dougl. ex |\n|Torr.                            |\n|Plant Symbol = VAME              |\n\nContributed by: USDA NRCS National Plant Data Center '
000006.doc
b'\nBefore the\n                      Federal Communication

In [2]:
def extract_text_from_df(file):
    text = None
    
    if not file['folder']:
        try:
            text = textract.process(file['path'])
        except:
            pass
        
    return text

In [3]:
df['raw_text'] = df.apply(lambda file: extract_text_from_df(file), axis=1)

NameError: name 'df' is not defined